### Load Dataset Files

In [0]:
%tensorflow_version 2.x

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
import cv2
from glob import glob
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential, optimizers
from tensorflow.keras.layers import Dense, BatchNormalization, Reshape, Dropout, Conv2D, MaxPool2D, Flatten, Input
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from tensorflow.keras import datasets, layers, models

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [3]:
tf.random.set_seed(42)
tf.__version__

'2.2.0'

In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
Content = "/content/gdrive/My Drive/new cnn/"

In [0]:
from zipfile import ZipFile
with ZipFile(Content+'train.zip', 'r') as z:
  z.extractall()

In [0]:
from zipfile import ZipFile
with ZipFile(Content+'test.zip', 'r') as z:
  z.extractall()

In [0]:
from zipfile import ZipFile
with ZipFile(Content+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

In [0]:
from zipfile import ZipFile
with ZipFile(Content+'labels.csv.zip', 'r') as z:
  z.extractall()

In [0]:
label = pd.read_csv("labels.csv")

In [11]:
label.head(5)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [12]:
label['breed'].value_counts()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
komondor                 67
brabancon_griffon        67
golden_retriever         67
eskimo_dog               66
briard                   66
Name: breed, Length: 120, dtype: int64

### Get one-hot encodings of labels

In [0]:
sample1 = pd.Series(label['breed'])
hot_enc = pd.get_dummies(sample1, sparse = True)
hot_labels = np.asarray(hot_enc)

In [14]:
hot_labels[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [15]:
img_rows = 128
img_cols = 128
num_channel = 1

X_train = []
Y_train = []

for f, img in tqdm(label.values):
  train_img = cv2.imread('./train/{}.jpg'.format(f),1)
  train_img_resize  = cv2.resize(train_img, (img_rows, img_cols))
  X_train.append(train_img_resize)
  Y_train.append(img)

100%|██████████| 10222/10222 [00:31<00:00, 326.47it/s]


In [0]:
# normalize to the range 0-1
X_train_data =np.array(X_train) / 255

In [0]:
train_X, val_X, train_Y, val_Y = train_test_split(X_train_data,hot_labels, test_size = 0.2, random_state = 345)

In [18]:
sample = pd.read_csv("sample_submission.csv")
sample.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,...,norwegian_elkhound,norwich_terrier,old_english_sheepdog,otterhound,papillon,pekinese,pembroke,pomeranian,pug,redbone,rhodesian_ridgeback,rottweiler,saint_bernard,saluki,samoyed,schipperke,scotch_terrier,scottish_deerhound,sealyham_terrier,shetland_sheepdog,shih-tzu,siberian_husky,silky_terrier,soft-coated_wheaten_terrier,staffordshire_bullterrier,standard_poodle,standard_schnauzer,sussex_spaniel,tibetan_mastiff,tibetan_terrier,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333

In [0]:
samp_img = sample['id']

In [20]:
x_test_feature = []
i = 0 
for f in tqdm (samp_img.values): 
    img = cv2.imread('./test/{}.jpg'.format(f), 0)
    img_resize = cv2.resize(img, (img_rows, img_cols)) 
    x_test_feature.append(img_resize)

100%|██████████| 10357/10357 [00:14<00:00, 692.32it/s]


In [0]:
# normalize to the range 0-1
test =np.array(x_test_feature) / 255

In [0]:
# Clear any previous model from memory
tf.keras.backend.clear_session()


In [0]:
 Initialize model
 model = Sequential()
 model.add(Conv2D(32, kernel_size = (5,5), strides = (1,1), activation='relu', input_shape=(128, 128, 3)))
 model.add(Conv2D(32, kernel_size = (3,3), strides = (1,1), activation='relu'))
 model.add(Flatten())
 model.add(Dense(256, activation='relu'))
 model.add(Dense(120, activation='softmax'))

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 124, 124, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 122, 122, 32)      9248      
_________________________________________________________________
flatten (Flatten)            (None, 476288)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               121929984 
_________________________________________________________________
dense_1 (Dense)              (None, 120)               30840     
Total params: 121,972,504
Trainable params: 121,972,504
Non-trainable params: 0
_________________________________________________________________


In [26]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=5, verbose=1) 

history = model.fit(train_X, train_Y, epochs= 10, verbose=1, validation_data=(val_X, val_Y), batch_size= 128, use_multiprocessing= True,callbacks = [es] )

Epoch 1/10
64/64 [==============================] - 6s 96ms/step - loss: 5.6930 - accuracy: 0.0104 - val_loss: 4.7868 - val_accuracy: 0.0068
Epoch 2/10
64/64 [==============================] - 6s 88ms/step - loss: 4.7856 - accuracy: 0.0120 - val_loss: 4.7862 - val_accuracy: 0.0068
Epoch 3/10
64/64 [==============================] - 6s 88ms/step - loss: 4.7838 - accuracy: 0.0138 - val_loss: 4.7863 - val_accuracy: 0.0064
Epoch 4/10
64/64 [==============================] - 6s 87ms/step - loss: 4.7623 - accuracy: 0.0202 - val_loss: 4.7415 - val_accuracy: 0.0166
Epoch 5/10
64/64 [==============================] - 6s 87ms/step - loss: 4.3562 - accuracy: 0.0947 - val_loss: 4.8300 - val_accuracy: 0.0240
Epoch 6/10
64/64 [==============================] - 6s 88ms/step - loss: 2.8659 - accuracy: 0.3643 - val_loss: 5.8047 - val_accuracy: 0.0274
Epoch 7/10
64/64 [==============================] - 6s 88ms/step - loss: 0.8493 - accuracy: 0.8126 - val_loss: 8.0445 - val_accuracy: 0.0235
Epoch 8/10
64

In [27]:
loss, accuracy = model.evaluate(train_X, train_Y, verbose=0)
print('Train Accuracy: %.3f' % accuracy)
print('Train Loss: %.3f' % loss)

val_loss, val_accuracy = model.evaluate(val_X, val_Y, verbose=0)
print('Validation Accuracy: %.3f' % val_accuracy)
print('Validation Loss: %.3f' % val_loss)

Train Accuracy: 0.999
Train Loss: 0.015
Validation Accuracy: 0.023
Validation Loss: 10.788
